# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1o2lpxoi9heyQV1hIlsHXWSfDkBPtze-V?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [12]:

from selenium import webdriver 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By


# Iniciando Options
chrome_options = Options()
#chrome_options.add_argument("--headless")
options = chrome_options

# Iniciando o chrome
nav = webdriver.Chrome(ChromeDriverManager().install(),options=options)


C:\Users\felip\AppData\Local\Temp\ipykernel_5900\1204829669.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  nav = webdriver.Chrome(ChromeDriverManager().install(),options=options)


In [18]:
nav.get("https://www.google.com/")
print(nav.find_element(By.NAME,'q').send_keys())


In [7]:
# pesquisar cotação dolar
nav.find_element('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar")
nav.find_element('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_dolar = nav.find_element('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_dolar)

# pesquisar cotação euro
nav.get("https://www.google.com/")
nav.find_element('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")
nav.find_element('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_euro = nav.find_element('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_euro)

# pegar cotacao ouro
nav.get("https://www.melhorcambio.com/")
aba_original = nav.window_handles[0]
nav.find_element('//*[@id="commodity-hoje"]/tbody/tr[2]/td[2]/a/img').click()
aba_nova = nav.window_handles[1]
nav.switch_to.window(aba_nova)
cotacao_ouro = nav.find_element('comercial').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(",", ".")
print(cotacao_ouro)

nav.quit()

InvalidArgumentException: Message: invalid argument: invalid locator
  (Session info: headless chrome=103.0.5060.134)
Stacktrace:
Backtrace:
	Ordinal0 [0x01135FD3+2187219]
	Ordinal0 [0x010CE6D1+1763025]
	Ordinal0 [0x00FE3E78+802424]
	Ordinal0 [0x01011DA1+990625]
	Ordinal0 [0x01011EAB+990891]
	Ordinal0 [0x0103EC92+1174674]
	Ordinal0 [0x0102CBD4+1100756]
	Ordinal0 [0x0103CFC2+1167298]
	Ordinal0 [0x0102C9A6+1100198]
	Ordinal0 [0x01006F80+946048]
	Ordinal0 [0x01007E76+949878]
	GetHandleVerifier [0x013D90C2+2721218]
	GetHandleVerifier [0x013CAAF0+2662384]
	GetHandleVerifier [0x011C137A+526458]
	GetHandleVerifier [0x011C0416+522518]
	Ordinal0 [0x010D4EAB+1789611]
	Ordinal0 [0x010D97A8+1808296]
	Ordinal0 [0x010D9895+1808533]
	Ordinal0 [0x010E26C1+1844929]
	BaseThreadInitThunk [0x7626FA29+25]
	RtlGetAppContainerNamedObjectPath [0x776D7A9E+286]
	RtlGetAppContainerNamedObjectPath [0x776D7A6E+238]


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [21]:
import pandas as pd
produtos_df = pd.read_csv('Produtos.csv')
display(produtos_df)

,Nomes
0,Felipe
1,Bruno
2,Moreira
3,Nunes


- Atualizando os preços e o cálculo do Preço Final

In [ ]:
produtos_df.loc[produtos_df['Moeda']=="Dólar", "Cotação"] = float(cotacao_dolar)
produtos_df.loc[produtos_df['Moeda']=="Euro", "Cotação"] = float(cotacao_euro)
produtos_df.loc[produtos_df['Moeda']=="Ouro", "Cotação"] = float(cotacao_ouro)

produtos_df['Preço Base Reais'] = produtos_df['Cotação'] * produtos_df['Preço Base Original']
produtos_df['Preço Final'] = produtos_df['Margem'] * produtos_df['Preço Base Reais']
# produtos_df["Preço Final"] = produtos_df["Preço Final"].map("{:.2f}".format)
display(produtos_df)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,4.914400,4914.350856,1.40,6880.091198
1,Carro Renault,4500.00,Euro,5.863321,26384.946732,2.00,52769.893464
2,Notebook Dell,899.99,Dólar,4.914400,4422.910856,1.70,7518.948455
3,IPhone,799.00,Dólar,4.914400,3926.605600,1.70,6675.229520
4,Carro Fiat,3000.00,Euro,5.863321,17589.964488,1.90,33420.932527
5,Celular Xiaomi,480.48,Dólar,4.914400,2361.270912,2.00,4722.541824
6,Joia 20g,20.00,Ouro,280.470000,5609.400000,1.15,6450.810000


### Agora vamos exportar a nova base de preços atualizada

In [ ]:
produtos_df.to_excel("Produtos Atualizado.xlsx", index=False)

In [18]:
nav.quit()